#1.**Introduction**

This project focuses on optimizing loyalty programs for retail supermarkets in Kenya using AI-driven techniques. As the retail landscape becomes increasingly competitive, supermarkets must leverage data to better understand customer behavior and personalize engagement strategies. By analyzing transaction data, customer demographics, and purchasing patterns, this project aims to design intelligent loyalty models that enhance customer retention, increase basket size, and drive long-term value. The integration of machine learning algorithms and predictive analytics will allow for dynamic reward systems and targeted promotions, tailored to the unique preferences and needs of Kenyan consumers.

#2.**Business** **Understanding**

## Problem Statement

Traditional loyalty programs fail to adapt to customer behavior, leading to:

- Generic campaigns with low engagement.
- Wasted promotional budgets.
- Declining customer retention.

##Objectives

•	Develop machine learning models to segment customers based on purchase behavior and loyalty metrics.

•	Predict customer churn and estimate lifetime value to proactively retain high-value customers.

•	Build a recommendation engine for personalized promotions to increase engagement and category-level spending.

•	Measure the impact of AI-driven personalization on retention, basket size, and promotional ROI.


#3.**Data Understanding**

•	The data preparation phase will involve cleaning and integrating multiple datasets, including transactional records, customer demographics, loyalty card activity, and promotional history. Key features such as recency, frequency, monetary value (RFM), category-level purchases, and redemption behavior will be engineered to enhance model accuracy. Missing values, inconsistent formats, and outliers will be addressed through standard preprocessing techniques. Data will be aggregated at both the customer and transaction level to support segmentation, prediction, and recommendation tasks

##Libraries

In [68]:
# Importing python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve,RocCurveDisplay,classification_report
%matplotlib inline

# Import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

##Import Datasets

In [69]:
# importing data from csv and
customer_df=pd.read_csv('customer_info.csv', encoding='ISO-8859-1')
transactions_df=pd.read_csv('POS_Transanctions.csv', encoding='ISO-8859-1')

In [70]:
customer_df.describe()

,loyalty_id,points
count,5.203700e+04,52037.000000
mean,2.203052e+07,1020.258277
std,1.673119e+07,1792.822970
min,1.000000e+00,0.000000
25%,9.016782e+06,154.000000
50%,1.700240e+07,443.000000
75%,3.424801e+07,1175.000000
max,6.700199e+07,163918.000000


In [71]:
transactions_df.describe()

,PAYMENT_TYPE,PRODUCT_ID,Unnamed: 16
count,1.022369e+06,1.034747e+06,0.0
mean,4.517483e+00,5.568171e+05,NaN
std,2.141840e+00,1.603369e+05,NaN
min,1.000000e+00,1.100010e+05,NaN
25%,2.000000e+00,4.075710e+05,NaN
50%,6.000000e+00,5.261230e+05,NaN
75%,6.000000e+00,7.000060e+05,NaN
max,6.000000e+00,9.899930e+05,NaN


In [72]:
customer_df.head()

,loyalty_id,first_name,city,country_code,ethnicity,gender,family_status,points,registered_date,registered_location,Customer_VAR1,national_id
0,1,ANTH,Nairobi,Kenya,Kenyan,Male,Married,1606.00,8/24/2015,RUAKA,Mrs.,34444130
1,4,ELIZ,Nairobi,Kenya,Kenyan,Female,Married,2921.56,9/7/2015,RUAKA,Mrs.,1827976
2,24,NGAN,Nairobi,Kenya,Kenyan,Male,Single,2412.00,10/17/2015,RUAKA,Mr.,24120172
3,32,EVA,Nairobi,Kenya,Kenyan,Female,Single,1281.00,11/5/2015,RUAKA,Mrs.,23883273
4,34,TIMO,Nairobi,Kenya,Kenyan,Male,Single,267.00,11/8/2015,RUAKA,Mr.,21853866


In [73]:
transactions_df.head()

,TICKET_ID,TICKET_DATE,PAYMENT_TYPE,PRODUCT_ID,DESCRIPTION,CATEGORY,DEPARTMENT,BRAND,REVENUE_VALUE,QTY_SOLD_UNIT,PURCHASE_PRICE,UNIT_PRICE,DISCOUNT,LOYALTY_ID,REDEEMED_POINTS,POINTS_EARNED,Unnamed: 16
0,QM0060236440087,10/1/2024,6.0,661047,HH-QM NON WOVEN PLAIN BAG V25 70GSM,TEXTILE,PACKAGING,QUICKMART,20,1,12,20,NaN,25012986,NaN,0,NaN
1,QM0060334900088,10/1/2024,6.0,661047,HH-QM NON WOVEN PLAIN BAG V25 70GSM,TEXTILE,PACKAGING,QUICKMART,20,1,12,20,NaN,9016420,NaN,0,NaN
2,QM0060536350045,10/1/2024,6.0,661047,HH-QM NON WOVEN PLAIN BAG V25 70GSM,TEXTILE,PACKAGING,QUICKMART,20,1,12,20,NaN,11008074,NaN,0,NaN
3,QM0060436550071,10/1/2024,6.0,661047,HH-QM NON WOVEN PLAIN BAG V25 70GSM,TEXTILE,PACKAGING,QUICKMART,20,1,12,20,NaN,9022233,NaN,0,NaN
4,QM0060334900114,10/1/2024,6.0,661047,HH-QM NON WOVEN PLAIN BAG V25 70GSM,TEXTILE,PACKAGING,QUICKMART,20,1,12,20,NaN,39013061,NaN,0,NaN


In [74]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52037 entries, 0 to 52036
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   loyalty_id           52037 non-null  int64  
 1   first_name           52037 non-null  object 
 2   city                 52037 non-null  object 
 3   country_code         52037 non-null  object 
 4   ethnicity            52037 non-null  object 
 5   gender               52037 non-null  object 
 6   family_status        52037 non-null  object 
 7   points               52037 non-null  float64
 8   registered_date      52037 non-null  object 
 9   registered_location  52037 non-null  object 
 10  Customer_VAR1        52037 non-null  object 
 11  national_id          51883 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 4.8+ MB


In [75]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034747 entries, 0 to 1034746
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   TICKET_ID          1034747 non-null  object 
 1   TICKET_DATE        1034747 non-null  object 
 2   PAYMENT_TYPE       1022369 non-null  float64
 3   PRODUCT_ID         1034747 non-null  int64  
 4   DESCRIPTION        1034747 non-null  object 
 5   CATEGORY           1034747 non-null  object 
 6   DEPARTMENT         1034747 non-null  object 
 7   BRAND              1034747 non-null  object 
 8    REVENUE_VALUE     1022369 non-null  object 
 9    QTY_SOLD_UNIT     1034747 non-null  object 
 10   PURCHASE_PRICE    1022369 non-null  object 
 11   UNIT_PRICE        1022369 non-null  object 
 12   DISCOUNT          168421 non-null   object 
 13  LOYALTY_ID         1034747 non-null  object 
 14   REDEEMED_POINTS   12378 non-null    object 
 15   POINTS_EARNED     1022369 non-n

#4.**Data Cleaning**

##Correct Formats

In [76]:
#Converting the columns into lower case for both datasets
customer_df.columns = customer_df.columns.str.lower()
transactions_df.columns = transactions_df.columns.str.lower()

In [77]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034747 entries, 0 to 1034746
Data columns (total 17 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ticket_id          1034747 non-null  object 
 1   ticket_date        1034747 non-null  object 
 2   payment_type       1022369 non-null  float64
 3   product_id         1034747 non-null  int64  
 4   description        1034747 non-null  object 
 5   category           1034747 non-null  object 
 6   department         1034747 non-null  object 
 7   brand              1034747 non-null  object 
 8    revenue_value     1022369 non-null  object 
 9    qty_sold_unit     1034747 non-null  object 
 10   purchase_price    1022369 non-null  object 
 11   unit_price        1022369 non-null  object 
 12   discount          168421 non-null   object 
 13  loyalty_id         1034747 non-null  object 
 14   redeemed_points   12378 non-null    object 
 15   points_earned     1022369 non-n

In [78]:
# Strip column names to avoid hidden whitespaces
transactions_df.columns = transactions_df.columns.str.strip()

In [79]:
#Converting the registered date column and ticket date to date and time format
customer_df['registered_date'] = pd.to_datetime(customer_df['registered_date'])
transactions_df['ticket_date'] = pd.to_datetime(transactions_df['ticket_date'])


In [80]:
customer_df['loyalty_id'] = customer_df['loyalty_id'].astype('object')


##Confirming Mising values

In [81]:
print(customer_df.isnull().sum())

loyalty_id               0
first_name               0
city                     0
country_code             0
ethnicity                0
gender                   0
family_status            0
points                   0
registered_date          0
registered_location      0
customer_var1            0
national_id            154
dtype: int64


In [82]:
print(transactions_df.isnull().sum())

ticket_id                0
ticket_date              0
payment_type         12378
product_id               0
description              0
category                 0
department               0
brand                    0
revenue_value        12378
qty_sold_unit            0
purchase_price       12378
unit_price           12378
discount            866326
loyalty_id               0
redeemed_points    1022369
points_earned        12378
unnamed: 16        1034747
dtype: int64


In [83]:
customer_df.shape

(52037, 12)

In [84]:
transactions_df.shape

(1034747, 17)

In [85]:
# Dropping unneccesary columns
drop_columns = ["redeemed_points", "unnamed: 16", "payment_type", "purchase_price"]
transactions_df = transactions_df.drop(drop_columns, axis =1)

In [86]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034747 entries, 0 to 1034746
Data columns (total 13 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   ticket_id      1034747 non-null  object        
 1   ticket_date    1034747 non-null  datetime64[ns]
 2   product_id     1034747 non-null  int64         
 3   description    1034747 non-null  object        
 4   category       1034747 non-null  object        
 5   department     1034747 non-null  object        
 6   brand          1034747 non-null  object        
 7   revenue_value  1022369 non-null  object        
 8   qty_sold_unit  1034747 non-null  object        
 9   unit_price     1022369 non-null  object        
 10  discount       168421 non-null   object        
 11  loyalty_id     1034747 non-null  object        
 12  points_earned  1022369 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(11)
memory usage: 102.6+ MB


In [87]:
#Convert quantity column to numeric
transactions_df['qty_sold_unit'] = pd.to_numeric(transactions_df['qty_sold_unit'], errors='coerce')

#Drop rows where quantity is <= 0
transactions_df = transactions_df[transactions_df['qty_sold_unit'] > 0]


In [88]:
transactions_df.shape

(989955, 13)

In [89]:
dropped_columns = ["ethnicity", "customer_var1", "national_id", "family_status", "city"]
customer_df = customer_df.drop(dropped_columns, axis =1)

##Merging the datasets

In [90]:

customer_loyalty_program = pd.merge(customer_df, transactions_df, on='loyalty_id', how='left')

In [91]:
customer_loyalty_program.head()

,loyalty_id,first_name,country_code,gender,points,registered_date,registered_location,ticket_id,ticket_date,product_id,description,category,department,brand,revenue_value,qty_sold_unit,unit_price,discount,points_earned
0,1,ANTH,Kenya,Male,1606.00,2015-08-24,RUAKA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,ELIZ,Kenya,Female,2921.56,2015-09-07,RUAKA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,NGAN,Kenya,Male,2412.00,2015-10-17,RUAKA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32,EVA,Kenya,Female,1281.00,2015-11-05,RUAKA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34,TIMO,Kenya,Male,267.00,2015-11-08,RUAKA,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
customer_loyalty_program.shape

(80237, 19)

##Exploratory Data Analysis(EDA)

#5.Data Preparation

#Modelling

## A. Collaborative Filtering( Matrix Factorization)

## B. Content-Based Filtering (TF-IDF + Cosine Similarity)

#Evaluation

##A. Collaborative Filtering (RMSE)

##B. Content-Based (Precision@K)

#Deployment